In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from NoiseKD import Teacher

First hurdle. the embedding layer.  It requires the ints as inputs, not one hot.  i need one hot.  i will also need to rewire the LLM i use to do the same.  hmm.  save the weights, load it into a model that has the same shapes, but accepts one hot arrays, not vocab_indexes.

In [3]:
class TransformerEncoder(nn.Module):
    def __init__(self, embedding_dim, num_heads, hidden_dim, num_layers, dropout):
        super(TransformerEncoder, self).__init__()
        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(embedding_dim, num_heads, hidden_dim, dropout),
            num_layers
        )

    def forward(self, x):
        return self.transformer(x)

class SimpleLanguageModel(nn.Module):
    def __init__(self, vocab_size, sequence_length, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout):
        super(SimpleLanguageModel, self).__init__()

        # Define the embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # Define the transformer encoder
        self.transformer_encoder = TransformerEncoder(embedding_dim, num_heads, hidden_dim, num_layers, dropout)
        
        self.fc1 = nn.Linear(sequence_length*embedding_dim, 1000)
        self.fc2 = nn.Linear(1000, 500)
        
        self.output_layer = nn.Linear(500, class_num)
        
        

    def forward(self, input_data):
        # Input_data is of shape (batch_size, sequence_length)
        # Apply embedding layer
        #print(input_data.shape)
        embedded = self.embedding(input_data)
        #print(embedded.shape)
        # Pass through the transformer encoder
        transformed = self.transformer_encoder(embedded)
        #print(transformed.shape) same as input duh: batch x sequence_length x embedding_dim
        flattened_tensor = transformed.view(-1,sequence_length*embedding_dim)
        f1 = nn.ReLU()(self.fc1(flattened_tensor))
        f2 = nn.ReLU()(self.fc2(f1))
        out = self.output_layer(f2)
        # Apply the output layer
        output = F.softmax(out,dim=1)

        return output
    
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

In [4]:
embedding_dim = 16
num_heads = 8
hidden_dim  = 11
num_layers = 2
dropout = 0.1
vocab_size = 80
class_num = vocab_size
batch_size = 50
sequence_length = 160

In [5]:
SLM = SimpleLanguageModel(vocab_size, sequence_length, embedding_dim, class_num, num_heads, hidden_dim, num_layers, dropout)

In [6]:
SLM

SimpleLanguageModel(
  (embedding): Embedding(80, 16)
  (transformer_encoder): TransformerEncoder(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
          )
          (linear1): Linear(in_features=16, out_features=11, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=11, out_features=16, bias=True)
          (norm1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
        (1): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=16, out_features=16, bias=True)
       

In [7]:
input_data = torch.randint(low=0, high=vocab_size, size=(batch_size, sequence_length))
print(input_data.shape)
SLM(input_data).shape

torch.Size([50, 160])


torch.Size([50, 80])

In [8]:
count_parameters(SLM)

3105922

In [9]:
teacher_slm = Teacher(SLM,(sequence_length,)) #don't specify batch!!

In [10]:
##some of these configs made for more diverse outputs in teachers:
config_args = {"dist_type" : "ints" ##worked well
                      , "gen_m" : vocab_size
                      , "gen_n" : 100
                      , "gen_epochs" : 100
                      , "gen_lr" : 0.001
                      , "random_shuffle" : 0.1
                      , "out_type" : "one-hot" }

config_args_high_epochs = {"dist_type" : "ints" ##okay, but not as well as config_args.
                      , "gen_m" : vocab_size
                      , "gen_n" : 1000
                      , "gen_epochs" : 100
                      , "gen_lr" : 0.001
                      , "random_shuffle" : 0.1
                      , "out_type" : "one-hot" }

config_args_higher_lr = {"dist_type" : "ints" ##lower was worse.  raise it. 0.003 looks great.  this is the best.
                      , "gen_m" : vocab_size
                      , "gen_n" : 2000
                      , "gen_epochs" : 50
                      , "gen_lr" :  0.003 ##0.003
                      , "random_shuffle" : 0.8
                      , "out_type" : "one-hot" }

config_args_less_data = {"dist_type" : "ints" ##worse
                      , "gen_m" : vocab_size
                      , "gen_n" : 500
                      , "gen_epochs" : 100
                      , "gen_lr" : 0.003
                      , "random_shuffle" : 0.1
                      , "out_type" : "one-hot" }

config_args_high_shuffle = {"dist_type" : "ints" ##one bar..
                      , "gen_m" : vocab_size
                      , "gen_n" : 10_000
                      , "gen_epochs" : 10
                      , "gen_lr" : 0.05
                      , "random_shuffle" : 0.9
                      , "out_type" : "one-hot" }

config_args_small_batch = {"dist_type" : "ints" ##this one was the first to do well.  not just one bar and the rest nearly zero.
                      , "gen_m" : vocab_size
                      , "gen_n" : 10_000
                      , "gen_epochs" : 10
                      , "gen_lr" : 0.05
                      , "random_shuffle" : 0.5
                      , "batch_size" : 10
                      , "out_type" : "one-hot" }

config_ab = {"dist_type" : "ints" ##worked well
                      , "gen_m" : vocab_size
                      , "gen_n" : 5000
                      , "gen_epochs" : 200
                      , "gen_lr" : 0.005
                      , "random_shuffle" : 0.0
                      , "out_type" : "one-hot" 
                      , "dist_type" : 'hetero'
                      , "alpha" : 1
                      , "beta" : 4} #maybe increase epochs?

In [11]:
#teacher_slm.configure(**config_ab) #this is dying.  might be time for colab!!
teacher_slm.load_state_dict('good_teacher.pth')

Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.FloatTensor instead (while checking arguments for embedding)
lets try ints!


In [19]:

args = { 'val_train' : "val"
                      , 'n' : 1000
                      , 'dist_type' : 'ints'
                      , 'm' : vocab_size
                      , 'std': 1.0
                      , 'batch_size' : 50
        }
teacher_slm.generate_data(**args)

In [20]:
teacher_slm.val_inputs.dtype,teacher_slm.val_inputs.shape

(torch.int64, torch.Size([1000, 160]))

In [21]:
args_float_int = { 'val_train' : "val"
                      , 'n' : 1000
                      , 'dist_type' : 'linearized_ints'
                      , 'm' : vocab_size
                      , 'std': 1.0
                      , 'batch_size' : 50
        }

teacher_slm.generate_data(**args_float_int)

In [22]:
teacher_slm.val_inputs.dtype, teacher_slm.val_inputs.shape

(torch.float32, torch.Size([1000, 160, 80]))

In [ ]:
teacher_slm.model.embedding.weight.detach().numpy() #use this to make the matrix, just linear? not sure.

In [ ]:
 #the code is right, i just need this to be a better dist.

In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# Assuming you have teacher_slm.val_inputs as a tensor of shape (1000, input_dim)
val_inputs = teacher_slm.val_inputs

# Create a DataLoader with a batch size
batch_size = 1
data_loader = DataLoader(TensorDataset(val_inputs), batch_size=batch_size, shuffle=False)

# Initialize an empty list to store the results
results = []

# Set your model to evaluation mode
teacher_slm.model.eval()

# Iterate through the data loader in batches
for batch in data_loader:
    batch_inputs = batch[0]  # Get batch inputs
    with torch.no_grad():
        # Forward pass through the model
        batch_outputs = teacher_slm.model(batch_inputs)
    # Calculate the argmax along the last axis (-1)
    #batch_argmax = torch.argmax(batch_outputs, dim=-1)
    # Append the results to the list
    results.append(batch_outputs)

# Concatenate the results to get the final tensor of shape (1000, output_dim)
final_results = torch.cat(results, dim=0)

# Print the final_results tensor
print(final_results)

In [ ]:
torch.all(final_results[15] == teacher_slm.model(teacher_slm.val_inputs[15]))

In [ ]:
teacher_slm.model(teacher_slm.val_inputs[0:39])

In [ ]:
teacher_slm.graph_dataset_dist(val_train = 'val')

In [ ]:
torch.all(teacher_slm.model(teacher_slm.in_test[50:100]) == teacher_slm.val_targets[50:100]) ##okay, this is weird.  this works
#in test is concatenated the same way as the outputs in KDNoise.  only difference.  doesn't appear to be different.  
torch.all(teacher_slm.model(teacher_slm.val_inputs[50:100]) == teacher_slm.val_targets[50:100]) ##yeah, val_inputs and in_test appear to be the same.
#torch.all(teacher_slm.model(teacher_slm.in_test[:49]) == teacher_slm.val_targets[:49]) ##this doesn't/.  huh???!!
#torch.all(teacher_slm.model(teacher_slm.in_test[:50]) == teacher_slm.model(teacher_slm.val_inputs[:50])) ##okay, these match too
#torch.all(teacher_slm.model(teacher_slm.in_test[223:409]) == teacher_slm.model(teacher_slm.val_inputs[223:409])) #these match.  

In [ ]:
teacher_slm.val_inputs.shape, teacher_slm.val_targets.shape

In [ ]:
torch.all(teacher_slm.model(teacher_slm.val_inputs[1]) ==  teacher_slm.val_targets[1]) ##torch.allclose doesn't work either

so, it looks like it is predicting correctly, but there is something wrong with the batch? they match when you pass in 50, but more, or not at exactly the 50 marks, it doesn't all match.  

In [ ]:
#added teacher_slm.in_test and teacher_slm.dataloader
teacher_slm.data_loader
x = None
y = None
for batch_samples in teacher_slm.data_loader:
    # Perform inference on each batch
    batch_outputs = teacher_slm.model(batch_samples[0])  # Assuming samples are in the first element of the batch
    print(batch_samples[0])
    x = batch_samples[0]
    print(batch_outputs)
    y = batch_outputs
    break

In [ ]:
#okay, the first 50 match.  do they 
torch.all(x == teacher_slm.val_inputs[0:50]),torch.all(y == teacher_slm.val_targets[0:50]),torch.all(teacher_slm.model(x) == y)

In [ ]:
y.shape

specify gen outputs.  we need one hots
i can do this better than chat gpt lol

In [ ]:
##this function works, it is in LABNET.  
def one_hot_last_dim(tensor_shape):
    num_classes = tensor_shape[-1]
    random_idx = np.random.randint(1, num_classes + 1, size=tensor_shape[:-1])
    zero_tensor = np.zeros(tensor_shape, dtype=int)
    last_dim_indices = np.arange(num_classes)
    zero_tensor[..., :, last_dim_indices] = (random_idx[..., np.newaxis] == last_dim_indices)
    return zero_tensor

In [ ]:
from torch.optim.lr_scheduler import LambdaLR
import math

warmup_steps = 60_000  # Adjust this as needed steps is total items passed through
total_steps = 200_000  # Adjust this as needed i'd like to calculate this....


# Define a learning rate scheduler with warmup
def lr_lambda(current_step):
    if current_step < warmup_steps:
        # During warmup, increase learning rate linearly
        return float(current_step) / float(max(1, warmup_steps))
    else:
        # After warmup, decrease learning rate using some schedule
        # You can use any LR schedule you prefer here
        # For example, you can use a learning rate schedule like CosineAnnealing
        return 0.5 * (1 + math.cos(math.pi * (current_step - warmup_steps) / (total_steps - warmup_steps)))



""" Training loop
for step in range(total_steps):
    optimizer.zero_grad()
    # Compute your loss and backpropagation here
    loss.backward()
    optimizer.step()
    scheduler.step()
"""

In [ ]:
#############non-repeating train
learning_rate = 0.001 #0.00025 this shit is slooooow
momentum = 0.95
val_batch_size = 10


num_epochs = 50
batches_per_epoch = 100
batch_size = 1000
data_per_batch = batch_size

criterion =  nn.MSELoss() #nn.KLDivLoss() #nn.CrossEntropyLoss()  #i think stick to mse for now.  this probs just needs lots of time to start learning.  like s4 lol.
optimizer = optim.Adam(student.parameters(), lr=learning_rate)
scheduler = LambdaLR(optimizer, lr_lambda=lr_lambda)
#optimizer = optim.SGD(student.parameters(), lr=learning_rate, momentum=momentum)
#no train data loader here.
val_data = list(zip(teacher_slm.val_inputs, teacher_slm.val_targets))
val_input_tensors = torch.stack([torch.Tensor(x[0]) for x in val_data])
val_target_tensors = torch.stack([torch.Tensor(x[1]) for x in val_data])
val_dataset = TensorDataset(val_input_tensors, val_target_tensors)
val_dataloader = DataLoader(val_dataset, batch_size=val_batch_size, shuffle=True)
accuracy_threshold = 0.5
print_every = 1 #its working, its just lots of data mama.
validation_every = 1


gen_args  = { 'val_train' : "train"
              , 'n' : data_per_batch
              , 'dist_type' : 'ints' #this is generating ints for inputs, the outputs are logits.  hmmmm
              , 'm' : slm_init_config['vocab_size']
              , 'std': 1.0
              , 'display_progress' : False
        }

losses = []  # List to store losses
accuracies = []

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

student = student.to(device)
for epoch in range(num_epochs):
    student.train()
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    for i in range(batches_per_epoch):
        optimizer.zero_grad()  # Zero the gradients

        teacher_slm.generate_data(**gen_args)
        data_e = list(zip(teacher_slm.train_inputs, teacher_slm.train_targets))
        input_tensors_e = torch.stack([torch.Tensor(x[0]) for x in data_e])
        target_tensors_e = torch.stack([torch.Tensor(x[1]) for x in data_e])
        dataset_e = TensorDataset(input_tensors_e, target_tensors_e)
        dataloader_e = DataLoader(dataset_e, batch_size=batch_size, shuffle=True)

        for input_batch_e, target_batch_e in dataloader_e:
            optimizer.zero_grad()  # Zero the gradients
            input_batch_e = input_batch_e.to(device)
            target_batch_e = target_batch_e.to(device)
            output = student(input_batch_e)  # Forward pass
            loss = criterion(output, target_batch_e)  # Compute the loss
            loss.backward()  # Backpropagation
            optimizer.step()  # Update the weights

            scheduler.step() ##this does the warmup step.  

            total_loss += loss.item()
            #add early stopping...
            #and validation at each step.
            # Calculate accuracy
            predictions = (output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
            correct_predictions += (predictions == target_batch_e).sum().item()
            total_samples += input_batch_e.size(0)

    # Print the average loss for this epoch
    avg_loss = total_loss / ( batches_per_epoch * data_per_batch)
    losses.append(avg_loss)

    accuracy = correct_predictions / total_samples
    accuracies.append(accuracy)

    if (epoch + 1) % print_every == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss}, Train Accuracy: {accuracy:.4f}')
        current_lr = optimizer.param_groups[0]['lr']
        print(current_lr)
    
    if (epoch + 1) % validation_every == 0:
        student.eval()

        total_val_samples = 0
        correct_val_predictions = 0

        with torch.no_grad():
            for val_input_batch, val_target_batch in val_dataloader:
                val_input_batch = val_input_batch.to(device)
                val_target_batch = val_target_batch.to(device)
                val_output = student(val_input_batch)
                val_predictions = (val_output > accuracy_threshold).float()  # Assuming a threshold of 0.5 for binary classification
                correct_val_predictions += (val_predictions == val_target_batch).sum().item()
                total_val_samples += val_input_batch.size(0)

        # Calculate validation accuracy
        val_accuracy = correct_val_predictions / total_val_samples

        # Print the validation accuracy for this epoch
        print(f'\t\tValidation Accuracy: {val_accuracy:.4f}')

        # Set the model back to training mode
        student.train()
